In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
weekend_Oct_list = [6, 7, 13, 14, 20, 21, 27, 28]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in weekend_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [16]:
len(link_day_minute_Oct_list)

23040

In [17]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 960)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [18]:
x[:,:2]

matrix([[ 319339.94812959,  322829.63254807],
        [ 321151.27358843,  324192.52603324],
        [ 604536.79414522,  603940.25714808],
        [ 646966.78253242,  642828.7893651 ],
        [ 166326.22657791,  161608.37802022],
        [ 208291.09582251,  212708.14898556],
        [ 383602.35894361,  374307.40173799],
        [ 381778.48335853,  374079.69081353],
        [ 358327.89456711,  362211.98946173],
        [ 368370.95311028,  360048.32229853],
        [ 399414.3896342 ,  415818.05830128],
        [ 326493.37968399,  319913.87093684],
        [  87465.76160775,   89628.65786679],
        [  91876.00413094,   95378.89130406],
        [ 151555.2050378 ,  147252.50112781],
        [ 139983.68038   ,  138143.4445973 ],
        [ 200588.27970907,  201475.87986933],
        [ 208319.58518245,  211885.66013314],
        [ 358150.25884312,  351634.68698475],
        [ 331092.07755642,  319369.37662986],
        [ 486770.31156146,  453599.39317856],
        [ 531017.37431378,  511818

In [19]:
np.size(x, 0), np.size(x, 1)

(24, 960)

In [20]:
np.size(A,0), np.size(A,1)

(24, 140)

In [21]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-01, 8e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -8.89884478e+06  0.00000000e+00  5.46e+03 9.16e+02  1.00e+06     0s
   1  -2.27421494e+06 -3.16664279e+06  1.20e+03 2.18e+02  2.69e+05     0s
   2  -9.45820230e+05 -2.96024562e+06  3.83e+01 6.67e+00  4.23e+04     0s
   3  -1.51161458e+06 -1.83646374e+06  1.60e+00 2.69e-01  5.9

In [22]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [23]:
lam_list

[6.583472719785615e-08,
 28780.237570665828,
 68167.36334467526,
 13593.18212211939,
 22358.596023403486,
 1.1532851356958073e-05,
 69928.27572361386,
 7.559488175080495e-08,
 2.3666003964290536e-09,
 38263.54958789865,
 2.3665997145274625e-09,
 2.3666001858352654e-09,
 2.366116778282102e-09,
 2.382056732368905e-09,
 1.503103370523523e-05,
 2.608606460125847e-09,
 4.228269786131046e-09,
 125256.98269334178,
 415146.1572485195,
 3.248132431202364e-05,
 5.5219191030451265e-06,
 15288.008606770914,
 119775.78445898702,
 0.00017642136971086648,
 23642.83890915836,
 14671.626357482046,
 6.51793127581515e-06,
 58316.08040130123,
 26153.927318763057,
 2.6090536620853355e-09,
 8.211169871261451e-05,
 4.231065133178558e-09,
 39399.47233293899,
 122895.86336133789,
 3.1954209139206874e-06,
 0.00014297753280450329,
 2.6090493543401857e-09,
 42931.37610476395,
 4.229601637857864e-09,
 80724.20134961963,
 21087.603780573754,
 0.0002060187317678608,
 123186.35936909133,
 2.6090599428511764e-09,
 670